In [1]:
import numpy as np
import pandas as pd
import os

from prediction_utils.pytorch_utils.metrics import (
    StandardEvaluator
)

In [2]:
grp_label_dict = {1: "Black women", 2: "White women", 3: "Black men", 4: "White men"}

args = {
    "experiment_name": "apr14_thr",
    "cohort_path": "/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts/cohort/all_cohorts.csv",
    "base_path": "/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts",
    "eval_fold": "test",
}
aggregate_path = os.path.join(
    args["base_path"], "experiments", args["experiment_name"], "performance", "all"
)

preds_path = os.path.join(aggregate_path, "predictions.csv")
preds = pd.read_csv(preds_path)
eval_df = preds.query('phase == "test"')

In [23]:
eval_df.model_id.unique()

array([0.001     , 0.00215443, 0.00464159, 0.01      , 0.02154435,
       0.04641589, 0.1       , 0.21544347, 0.46415888, 1.        ])

In [26]:
preds_all = []
eqodds_threshold=0.1
for experiment in ['original_pce', 'revised_pce', 'apr14_erm', 'apr14_erm_recalib', 'apr14_mmd']:
    aggregate_path = os.path.join(args['base_path'], 'experiments', 
                                  experiment, 'performance',
                                  'all')
    preds_path = os.path.join(aggregate_path, 'predictions.csv')

    preds = pd.read_csv(preds_path)
    if 'model_id' not in preds.columns:
        preds = preds.assign(model_id=0)
    if 'fold_id' not in preds.columns:
        preds = preds.assign(fold_id=0)
    if experiment in ['apr14_mmd', 'apr14_thr']:
        preds = preds.query('model_id == @eqodds_threshold')
        
    preds_all.append(preds) 
preds_all = pd.concat(preds_all)
    #discovery_omission = get_false_discovery_omission(preds)
#     df_general = pd.read_csv(os.path.join(aggregate_path, 'standard_evaluation.csv'))
#     df_fair = pd.read_csv(os.path.join(aggregate_path, 'fairness_evaluation.csv'))

#     thr_calibs = pd.read_csv(os.path.join(aggregate_path, 'calibration', calib_method, 'calibration_sensitivity_thresholds_raw.csv'))
    
#     thr_calibs = (thr_calibs
#                   .assign(metric = lambda x: 'outcome_rate_' + x.pred_probs.map({0.075: '075', 0.2: '20'}))
#                   .rename(columns = {'calibration_density': 'performance'})
#                   .filter(df_fair.columns)
#                  )

#     plot_df = (df_general
#                .append(df_fair)
#                .append(thr_calibs)
#               )

#     if experiment in ['apr14_mmd', 'apr14_thr']:
#         plot_df = plot_df.query('model_id == @eqodds_threshold')
    
#     if 'model_id' not in plot_df.columns:
#         plot_df = plot_df.assign(model_id=0)
#     if 'fold_id' not in plot_df.columns:
#         plot_df = plot_df.assign(fold_id=0)
        

#     plot_df = (plot_df
#                     .set_index(['model_id','fold_id'], drop=True)
#                     .filter(['metric', 'performance_overall', 'performance', 'violation', 'group'])
#                     .drop_duplicates()
#                     .reset_index()
#                #.append(discovery_omission)
#                .assign(model_type = experiment)
#     )  
    
     
# plot_df_all = pd.concat(plot_df_all)
# plot_df_all = plot_df_all.assign(model_type = lambda x: x.model_type.map({'original_pce': 'PCE',
#                                                                           'revised_pce': 'revPCE',
#                              'apr14_erm': 'Base',
#                              'apr14_erm_recalib': 'recBase',
#                              'apr14_mmd': 'EqOdd'}))

In [32]:
preds_all.query("phase=='test'").groupby(['phase', 'model_type', 'fold_id', 'group']).count()

pred_probs  labels  weights  person_id  \
phase model_type  fold_id group                                           
test  eqodds_mmd  1       1             486     486      486        486   
                          2             808     808      808        808   
                          3             337     337      337        337   
                          4             701     701      701        701   
                  2       1             486     486      486        486   
...                                     ...     ...      ...        ...   
      recalib_erm 10      4             701     701      701        701   
      revised_pce 0       1             539     539      539        539   
                          2             856     856      856        856   
                          3             382     382      382        382   
                          4             757     757      757        757   

                                 treat  ldlc  relative_risk  model_id  \
phase model_type  fold_id group                                         
test  eqodds_mmd  1       1        486   486            486       486   
                          2        808   808            808       808   
                          3        337   337            337       337   
                          4        701   701            701       701   
                  2       1        486   486            486       486   
...                                ...   ...            ...       ...   
      recalib_erm 10      4        701   701            701       701   
      revised_pce 0       1        539   539            539       539   
                          2        856   856            856       856   
                          3        382   382            382       382   
                          4        757   757            757       757   

                                 outputs  config_id  
phase model_type  fold_id group                      
test  eqodds_mmd  1       1          486        486  
                          2          808        808  
                          3          337        337  
                          4          701        701  
                  2       1          486        486  
...                                  ...        ...  
      recalib_erm 10      4          701        701  
      revised_pce 0       1            0          0  
                          2            0          0  
                          3            0          0  
                          4            0          0  

[128 rows x 10 columns]

In [35]:
evaluator = StandardEvaluator(metrics=['auc', 'auprc', 'ace_rmse_logistic_log'],
                             threshold_metrics = ['sensitivity', 'specificity'],
                             thresholds = [0.075, 0.2])

# threshold_metrics = config_dict['logging_threshold_metrics'],
#                               thresholds = config_dict['logging_thresholds'],
#                               metrics = ['auc', 'auprc', 'loss_bce', 
#                                          'ace_rmse_logistic_log',
#                                          'ace_abs_logistic_log']
        
result_df_ci = evaluator.bootstrap_evaluate(
    df=preds_all.query("phase=='test'"),
    n_boot=1000,
    strata_vars_eval=['phase', 'model_type', 'fold_id', 'group'],
    strata_vars_boot=['phase', 'labels', 'group'],
    strata_var_replicate='fold_id',
    replicate_aggregation_mode=None,
    #strata_var_experiment='config_id',
    baseline_experiment_name=0,
    strata_var_group='group',
    weight_var='weights',
    n_jobs=-1,
    compute_overall=True,
)

In [37]:
aggregate_path_all = '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts/experiments/bmj_manuscript/'
os.makedirs(aggregate_path_all, exist_ok=True)

In [39]:
ls /labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts/experiments/bmj_manuscript/

bootstrap_standard_eval.csv


In [ ]:
result_df_ci.to_csv(os.path.join(aggregate_path_all, 'bootstrap_standard_eval.csv'), index=False)

In [20]:
result_df_ci.shape

(750, 9)

In [ ]:
evaluator = StandardEvaluator(metrics=['auc', 'loss_bce'])
result_df_ci = evaluator.bootstrap_evaluate(
    df=eval_df,
    n_boot=10,
    strata_vars_eval=['phase', 'model_type', 'fold_id', 'config_id', 'group'],
    strata_vars_boot=['phase', 'labels', 'group'],
    strata_var_replicate='fold_id',
    replicate_aggregation_mode=None,
    strata_var_experiment='config_id',
    baseline_experiment_name=0,
    strata_var_group='group',
    weight_var='weights',
    compute_overall=True,
    group_overall_name=-1,
)

In [ ]:
result_df_ci.group.unique()